In [26]:
%load_ext autoreload
%autoreload 2
import json
import src.job_scrape_funcs as funcs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Scrape job listings:
jobs = funcs.scrape_jobs(pages=2, job_title="Machine learning engineer", location="Chicago", post_time=30)

10 scraped successfully.
20 scraped successfully.
Making sure all descriptions scraped...
Finished!


In [28]:
jobs

[{'Title': 'Business Data Scientist, AI Analytics',
  'Company': 'Google',
  'Link': 'https://www.linkedin.com/jobs/view/business-data-scientist-ai-analytics-at-google-4189314893?position=1&pageNum=0&refId=1xpXKH3f%2BFKNVvhEE1zMCg%3D%3D&trackingId=Rri4CfUEWUBKFey5YAWW2g%3D%3D',
  'Posted time': '1 week ago',
  'Description': "This role may also be located in our Playa Vista, CA campus.Note: By applying to this position you will have an opportunity to share your preferred working location from the following: Seattle, WA, USA; Ann Arbor, MI, USA; Atlanta, GA, USA; Austin, TX, USA; Chicago, IL, USA; Mountain View, CA, USA; New York, NY, USA; Los Angeles, CA, USA; San Francisco, CA, USA; Washington D.C., DC, USA.Minimum qualifications:Master's degree in a quantitative discipline such as Statistics, Engineering, Sciences, or equivalent practical experience.3 years of experience using analytics to solve product or business problems, coding (e.g., Python, R, SQL), querying databases or statis

In [12]:
# Save to JSON:
with open("data\job_data_4.json", 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

<>:2: SyntaxWarning: invalid escape sequence '\j'
<>:2: SyntaxWarning: invalid escape sequence '\j'
C:\Users\tempu\AppData\Local\Temp\ipykernel_12800\4073811815.py:2: SyntaxWarning: invalid escape sequence '\j'
  with open("data\job_data_4.json", 'w', encoding='utf-8') as f:


In [ ]:
import fasttext
import fasttext.util
import pandas as pd

ft = fasttext.load_model('C:/Users/tempu/Documents/Code/job_tools/data/fasttext/cc.en.300.bin')
df = pd.read_json("data/job_data_3.json")
resume = open("data/resume.txt", "r").read()

In [7]:
df["Embedding"] = df["Description"].map(ft.get_word_vector)
resume_embed = ft.get_word_vector(resume)

In [8]:

from numpy import dot
from numpy.linalg import norm
def similarity(b):
    a = resume_embed
    return dot(a, b)/(norm(a)*norm(b))

In [9]:
df["Similarity"] = df["Embedding"].map(similarity)

In [11]:
df["Description"][497]

'Neptune Medical, is an innovative, venture capital funded medical device startup headquartered in Burlingame, CA. We have already commercialized novel medical devices using our technology. Now the team at Neptune is building something truly unique: a fully flexible medical robot that enables doctors to reach deep into human anatomy. Our Triton robot will disrupt flexible endoscopy and set a new standard with Triton Robotic Endoscopy in outcomes for decades to come.TITLE: Senior/Staff Software Engineer – Vision PlatformPOSITION SUMMARY:The Senior/Staff Software Engineer is a key member of the software team who will be responsible for implementing a robust vision compute pipeline and will work as a bridge between the software and the vision teams. As part of the software team responsible for the compute platform for the Triton robot, this candidate will be instrumental in the selection, evaluation and integration of the embedded Linux computers used in the robot. This member is expected

In [10]:
df.sort_values(by=["Similarity"], ascending=False)

,Title,Company,Link,Posted time,Description,Embedding,Similarity
404,"Software Engineer II, Machine Learning Platform",Attentive,https://www.linkedin.com/jobs/view/software-en...,2 days ago,Attentive® is the AI-powered mobile marketing ...,"[-0.0004491569, -0.0048827357, -0.0062222984, ...",0.981856
336,"Machine Learning Engineer, RAG",Salesforce,https://www.linkedin.com/jobs/view/machine-lea...,2 weeks ago,"To get the best candidate experience, please c...","[-0.0017611719, -0.004566508, -0.0060244976, 0...",0.981720
37,"Machine Learning Engineer, RAG",Slack,https://www.linkedin.com/jobs/view/machine-lea...,1 week ago,"To get the best candidate experience, please c...","[-0.001747923, -0.004586937, -0.005999878, 0.0...",0.981652
311,Software Engineer,Agave,https://www.linkedin.com/jobs/view/software-en...,1 year ago,tl;dr: we're looking for a world-class full-st...,"[0.00016293384, -0.0037459084, -0.0067062215, ...",0.981635
217,Machine Learning Engineer - New Verticals - Se...,DoorDash,https://www.linkedin.com/jobs/view/machine-lea...,1 week ago,About The TeamCome help us build the world's m...,"[-0.00042348227, -0.0051580123, -0.006993274, ...",0.981526
...,...,...,...,...,...,...,...
205,Software Engineer (Backend AI/ML),Lawrence Harvey,https://www.linkedin.com/jobs/view/software-en...,6 days ago,Software Engineer (Backend/ML) - Early Stage S...,"[0.00046328592, -0.006844704, -0.008118296, 0....",0.827534
486,System Software Engineer,Weave Robotics,https://www.linkedin.com/jobs/view/system-soft...,1 month ago,"Weave is making Isaac, a general-purpose, pers...","[0.0027742994, -0.008183243, -0.011471473, 0.0...",0.806703
475,Founding Machine Learning Engineer,Mineflow (YC S24),https://www.linkedin.com/jobs/view/founding-ma...,6 months ago,Roles And ResponsibilitiesDesign and implement...,"[0.002962639, -0.006857138, -0.0088345045, 0.0...",0.797352
335,Software Engineer - Frontend,Akhenaten,https://www.linkedin.com/jobs/view/software-en...,1 month ago,About UsAkhenaten is an early-stage cybersecur...,"[0.0050273556, -0.009857972, -0.0055191373, 0....",0.762359


In [13]:
df["Link"][404]

'https://www.linkedin.com/jobs/view/software-engineer-ii-machine-learning-platform-at-attentive-4175893099?position=5&pageNum=40&refId=0wiz909Irem%2B%2BXLx7MmWNw%3D%3D&trackingId=uznldakvMa%2FMyCdY1U2aPw%3D%3D'